# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [260]:
problem = '''Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.  
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment. 

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01. 
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r.'''

## 2. Ask for parameters

In [261]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [262]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Sets:
*  **S:** Set of investment strategies (1, 2, ..., 10)
*  **Ω:** Set of budget scenarios (1, 2, 3, 4)

## Parameters:

*  **R<sub>is</sub>:** Historical return of strategy *s* in period *i* (obtained from "investments_data.csv")
*  **A<sub>s</sub>:** Cost of investing in strategy *s*  ( [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] ) 
*  **B<sub>ω</sub>:** Available budget in scenario *ω* ([1000, 1100, 900, 1200])
*  **P<sub>ω</sub>:** Probability of budget scenario *ω* ([0.55, 0.4, 0.04, 0.01])
*  **α:** Tolerable probability of exceeding the budget (0.4)
*  **r:** Risk parameter (2)

## Decision Variables:

*  **x<sub>s</sub>:** Binary variable equal to 1 if investment strategy *s* is selected, 0 otherwise 



# 2. Ask for objective

In [263]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [264]:
Markdown(response2.text)

Minimize:  $\sum_{i} \frac{1}{|I|} (\sum_{s \in S} R_{is}x_s - \sum_{i} \frac{1}{|I|} \sum_{s \in S} R_{is}x_s)^2   + r * (\sum_{s \in S} A_s * x_s - \sum_{\omega \in \Omega} P_{\omega}B_{\omega})^2$ 


# 3. Ask for constraints

In [265]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [266]:
Markdown(response3.text)

## Constraints:

1. **Budget Constraint:** The total cost of selected investments cannot exceed the budget in any scenario with a probability higher than α.
   
   $  P \{ \sum_{s \in S} A_s * x_s \leq B_{\omega} \} \geq 1 - \alpha   \qquad  \forall \omega \in \Omega $

2. **Investment Uniqueness:** Each investment strategy can be selected at most once.

   $ x_s \in \{0,1\}  \qquad \forall s \in S $ 


## 3. Generate the pyomo code

In [267]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [268]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from "investments_data.csv")
data = {
    'Period': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
    'Strategy': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
    'Return': [0.02, 0.05, -0.01, 0.03, 0.04, 0.06, -0.02, 0.01, 0.03, 0.08, -0.03, 0.02]
}
df_returns = pd.DataFrame(data)

# Sets
S = range(1, 11)  # Investment strategies
Omega = range(1, 5)  # Budget scenarios

# Parameters
A = {s: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225][s - 1] for s in S}  # Cost of investment
B = {w: [1000, 1100, 900, 1200][w - 1] for w in Omega}  # Available budget
P = {w: [0.55, 0.4, 0.04, 0.01][w - 1] for w in Omega}  # Probability of budget scenario
alpha = 0.4  # Tolerable probability of exceeding the budget
r = 2  # Risk parameter

# Create a dictionary for historical returns R[i, s]
R = {(row.Period, row.Strategy): row.Return for row in df_returns.itertuples()}

# Model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(S, within=pyo.Binary)  # Investment decision

# Objective Function
def objective_rule(model):
    I = range(1, len(df_returns['Period'].unique()) + 1)  # Set of periods
    return sum(
        (1 / len(I)) * sum(R[i, s] * model.x[s] for s in S) -
        (1 / len(I)) * sum(R[j, s] * model.x[s] for s in S) for i in I for j in I
    ) ** 2 + r * (sum(A[s] * model.x[s] for s in S) - sum(P[w] * B[w] for w in Omega)) ** 2
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.budget_constraint = pyo.ConstraintList()
for w in Omega:
    model.budget_constraint.add(
        expr=sum(A[s] * model.x[s] for s in S) <= B[w] + (1 - P[w]) / P[w] * (sum(A[s] for s in S))
    )

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Objective:", model.objective())
for s in S:
    if pyo.value(model.x[s]) > 0.5:
        print(f"Invest in strategy {s}")
```

**Explanation:**

1. **Import Libraries:** Import the necessary libraries, including `pyomo` for optimization modeling and `pandas` for data manipulation.

2. **Data Preparation:**
   - Load the historical returns data from the "investments_data.csv" file using `pandas`.
   - Define other parameters (A, B, P, alpha, r) based on the provided information.
   - Prepare the return data (R) in a dictionary format for easier access in the model.

3. **Model Creation:** Create a concrete model using `pyo.ConcreteModel()`.

4. **Decision Variables:** Define the binary decision variable `x[s]` representing whether to invest in strategy *s* or not.

5. **Objective Function:** Formulate the objective function based on the provided formula, calculating the variance of returns and the penalty for exceeding the budget.

6. **Constraints:**
   - Implement the budget constraint using the probabilistic formulation provided.
   - Add a constraint to ensure that each investment strategy can be selected at most once.

7. **Solver:** Choose a solver (e.g., 'cbc', 'gurobi', etc.) using `pyo.SolverFactory`.

8. **Solution:** Solve the model using the selected solver and store the results.

9. **Output:** Print the solver's status, the optimal objective value, and the selected investment strategies.

Remember to replace the sample data for `R` with your actual data from the "investments_data.csv" file.

## 4. Input problem data and try running the generated code

In [269]:
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from "investments_data.csv")
data = {
    'Period': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
    'Strategy': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
    'Return': [0.02, 0.05, -0.01, 0.03, 0.04, 0.06, -0.02, 0.01, 0.03, 0.08, -0.03, 0.02]
}
df_returns = pd.DataFrame(data)

# Sets
S = range(1, 11)  # Investment strategies
Omega = range(1, 5)  # Budget scenarios

# Parameters
A = {s: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225][s - 1] for s in S}  # Cost of investment
B = {w: [1000, 1100, 900, 1200][w - 1] for w in Omega}  # Available budget
P = {w: [0.55, 0.4, 0.04, 0.01][w - 1] for w in Omega}  # Probability of budget scenario
alpha = 0.4  # Tolerable probability of exceeding the budget
r = 2  # Risk parameter

# Create a dictionary for historical returns R[i, s]
R = {(row.Period, row.Strategy): row.Return for row in df_returns.itertuples()}

# Model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(S, within=pyo.Binary)  # Investment decision

# Objective Function
def objective_rule(model):
    I = range(1, len(df_returns['Period'].unique()) + 1)  # Set of periods
    return sum(
        (1 / len(I)) * sum(R[i, s] * model.x[s] for s in S) -
        (1 / len(I)) * sum(R[j, s] * model.x[s] for s in S) for i in I for j in I
    ) ** 2 + r * (sum(A[s] * model.x[s] for s in S) - sum(P[w] * B[w] for w in Omega)) ** 2
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.budget_constraint = pyo.ConstraintList()
for w in Omega:
    model.budget_constraint.add(
        expr=sum(A[s] * model.x[s] for s in S) <= B[w] + (1 - P[w]) / P[w] * (sum(A[s] for s in S))
    )

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Objective:", model.objective())
for s in S:
    if pyo.value(model.x[s]) > 0.5:
        print(f"Invest in strategy {s}")

ERROR: Rule failed when generating expression for Objective objective with
index None: KeyError: (1, 5)
ERROR: Constructing component 'objective' from data=None failed:
        KeyError: (1, 5)


KeyError: (1, 5)

## 5. Correct the code to verify model viability (optional)

In [283]:
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data from "investments_data.csv")
# data = {
#     'Period':  [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
#     'Strategy': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
#     'Return': [0.02, 0.05, -0.01, 0.03, 0.04, 0.06, -0.02, 0.01, 0.03, 0.08, -0.03, 0.02]
# }
# df_returns = pd.DataFrame(data)


def transform_investment_data(file_path):
    # Read the CSV file with the specified delimiter
    df = pd.read_csv(file_path, delimiter=';')
    
    # Get the number of strategies (columns in the CSV file)
    num_strategies = df.shape[1]
    
    # Prepare the data dictionary
    data = {
        'Period': [],
        'Strategy': [],
        'Return': []
    }
    
    # Populate the data dictionary
    for period in range(len(df)):
        for strategy in range(num_strategies):
            data['Period'].append(period + 1)
            data['Strategy'].append(strategy + 1)
            data['Return'].append(df.iloc[period, strategy])
    
    # Create the DataFrame
    df_returns = pd.DataFrame(data)
    
    return df_returns

# Example usage:
file_path = 'investments_data.csv'
df_returns = transform_investment_data(file_path)

# Sets
S = range(1, 11)  # Investment strategies
Omega = range(1, 5)  # Budget scenarios

# Parameters
A = {s: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225][s - 1] for s in S}  # Cost of investment
B = {w: [1000, 1100, 900, 1200][w - 1] for w in Omega}  # Available budget
P = {w: [0.55, 0.4, 0.04, 0.01][w - 1] for w in Omega}  # Probability of budget scenario
alpha = 0.4  # Tolerable probability of exceeding the budget
r = 2  # Risk parameter

# Create a dictionary for historical returns R[i, s]
R = {(row.Period, row.Strategy): row.Return for row in df_returns.itertuples()}

# Model
model = pyo.ConcreteModel()

# Variables
model.x = pyo.Var(S, within=pyo.Binary)  # Investment decision

# Objective Function
def objective_rule(model):
    I = range(1, len(df_returns['Period'].unique()) + 1)  # Set of periods
    return sum(
        (1 / len(I)) * sum(R[i, s] * model.x[s] for s in S) -
        (1 / len(I)) * sum(R[j, s] * model.x[s] for s in S) for i in I for j in I
    ) ** 2 + r * (sum(A[s] * model.x[s] for s in S) - sum(P[w] * B[w] for w in Omega)) ** 2
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.budget_constraint = pyo.ConstraintList()
for w in Omega:
    model.budget_constraint.add(
        expr=sum(A[s] * model.x[s] for s in S) <= B[w] + (1 - P[w]) / P[w] * (sum(A[s] for s in S))
    )

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Objective:", model.objective())
for s in S:
    if pyo.value(model.x[s]) > 0.5:
        print(f"Invest in strategy {s}")


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 10
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.039338111877441406
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective: 2.5063436008950844e-16
Invest in strategy 1
Invest in strategy 2
Invest in strategy 3
Invest in strategy 4
Invest in strategy 5
Invest in strategy 6
Invest in strategy 7
Invest in strategy 8
Invest in strategy 9
Invest in strategy 10


## 6. Print the responses

In [256]:
print(response.text)

## Variables:

* **X1:**  Number of 1.5L COLI units produced daily. 
* **X2:** Number of 1.5L PEPSA units produced daily.
* **Y:** Hectoliters of syrup purchased and processed daily. 



In [257]:
print(response2.text)

Maximize:  49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * Y 



In [258]:
print(response3.text)

## Constraints:

1. **Syrup Conversion:** 
   *  X1 <= 40 * Y  (Maximum COLI units from Y hectoliters of syrup)
   *  X2 <= 20 * Y  (Maximum PEPSA units from Y hectoliters of syrup) 

2. **Production Line Capacity:**
   *  X1 + X2 <= 7100 * 8  (Total units produced daily cannot exceed line capacity)

3. **Minimum PEPSA Production:**
   *  X2 >= 0.5 * X1 (At least half the amount of COLI must be PEPSA)

4. **Non-Negativity:**
   *  X1 >= 0 
   *  X2 >= 0
   *  Y >= 0 



In [259]:
print(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.X1 = Var(domain=NonNegativeReals)  # Number of 1.5L COLI units produced daily
model.X2 = Var(domain=NonNegativeReals)  # Number of 1.5L PEPSA units produced daily
model.Y = Var(domain=NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define objective function
def objective_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * model.Y
model.profit = Objective(rule=objective_rule, sense=maximize)

# Define constraints
model.syrup_coli = Constraint(expr=model.X1 <= 40 * model.Y)
model.syrup_pepsa = Constraint(expr=model.X2 <= 20 * model.Y)
model.production_capacity = Constraint(expr=model.X1 + model.X2 <= 7100 * 8)
model.min_pepsa = Constraint(expr=model.X2 >= 0.5 * model.X1)

# Solve the model
solver = SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# 